In [1]:
import os
import pandas as pd

# Direct Mail inputs folder path
dm_folder_path = "C:\\Users\\pskotte\\Desktop\\New folder"
dm_files = [os.path.join(dm_folder_path, file) for file in os.listdir(dm_folder_path) if file.endswith('.csv')]
df_dm = pd.concat((pd.read_csv(file) for file in dm_files), ignore_index=True)

# ILMS inputs folder path
ilms_folder_path = "C:\\Users\\pskotte\\Desktop\\ILMS"
ilms_files = [os.path.join(ilms_folder_path, file) for file in os.listdir(ilms_folder_path) if file.endswith('.csv')]
df_ilms = pd.concat((pd.read_csv(file) for file in ilms_files), ignore_index=True)

print("DataFrames created for DM and ILMS.")

DataFrames created for DM and ILMS.


In [2]:
from datetime import datetime, timedelta
import pandas as pd

# Assuming df_dm and df_ilms are the DataFrames for Direct Mail and ILMS respectively

# Calculate the date 30 days ago from today
date_30_days_ago = datetime.now() - timedelta(days=30)

# Convert 'Report Date' columns to datetime if not already
df_dm['Report Date'] = pd.to_datetime(df_dm['Report Date'], errors='coerce')
df_ilms['Report Date'] = pd.to_datetime(df_ilms['Report Date'], errors='coerce')

# Filter rows for the last 30 days including today
df_dm_last_month = df_dm[df_dm['Report Date'] >= date_30_days_ago]
df_ilms_last_month = df_ilms[df_ilms['Report Date'] >= date_30_days_ago]

C:\Users\pskotte\AppData\Local\Temp\ipykernel_15284\3907528052.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_dm['Report Date'] = pd.to_datetime(df_dm['Report Date'], errors='coerce')
C:\Users\pskotte\AppData\Local\Temp\ipykernel_15284\3907528052.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ilms['Report Date'] = pd.to_datetime(df_ilms['Report Date'], errors='coerce')


In [3]:
from datetime import datetime, timedelta
import pandas as pd

# Calculate the cost for Returned/NOC items for the last 30 days. All items have a base cost, except for "REDEPOSITED", which has a different cost.

last_30_days = datetime.now() - timedelta(days=30)

# Ensure that 'Report Date' is in datetime format for comparison
df_dm['Report Date'] = pd.to_datetime(df_dm['Report Date'])
df_ilms['Report Date'] = pd.to_datetime(df_ilms['Report Date'])

# Filter for the last 30 days
df_dm_last_30_days = df_dm[(df_dm['Report Date'] > last_30_days) & (df_dm['Report Date'] <= datetime.now())]
df_ilms_last_30_days = df_ilms[(df_ilms['Report Date'] > last_30_days) & (df_ilms['Report Date'] <= datetime.now())]

# Calculate the cost based on the Return/NOC Reason column.
df_dm_last_30_days['DM Returned Item Cost'] = df_dm_last_30_days.apply(lambda x: 0.5 if x['Return/NOC Reason'] == 'REDEPOSITIED' else 1.2, axis=1)
df_ilms_last_30_days['ILMS Returned Item Cost'] = df_ilms_last_30_days.apply(lambda x: 0.5 if x['Return/NOC Reason'] == 'REDEPOSITIED' else 1.2, axis=1)

# Group by 'Report Date' and sum 'Returned Item Cost' for each day
df_dm_daily_cost = df_dm_last_30_days.groupby(df_dm_last_30_days['Report Date'].dt.date).agg({'DM Returned Item Cost': 'sum'})
df_ilms_daily_cost = df_ilms_last_30_days.groupby(df_ilms_last_30_days['Report Date'].dt.date).agg({'ILMS Returned Item Cost': 'sum'})

print("Daily Returned Items Cost for the Last 30 Days for Direct Mail:")
print(df_dm_daily_cost)
print("\nDaily Returned Items Cost for the Last 30 Days for ILMS:")
print(df_ilms_daily_cost)

Daily Returned Items Cost for the Last 30 Days for Direct Mail:
             DM Returned Item Cost
Report Date                       
2024-03-18                  2122.8
2024-03-19                  8236.8
2024-03-20                 13376.4
2024-03-21                  3872.4
2024-03-22                  4797.6
2024-03-25                  7395.6
2024-03-26                  7976.4
2024-03-27                  4978.8
2024-03-28                  3313.2
2024-03-29                  4032.0
2024-04-01                  8958.0
2024-04-02                 16078.8
2024-04-03                 10072.8
2024-04-04                  5034.0
2024-04-05                  5626.8
2024-04-08                  6127.2
2024-04-09                  7711.2
2024-04-10                  4488.0
2024-04-11                  3068.4
2024-04-12                  3674.4
2024-04-15                  5450.4

Daily Returned Items Cost for the Last 30 Days for ILMS:
             ILMS Returned Item Cost
Report Date                         

C:\Users\pskotte\AppData\Local\Temp\ipykernel_15284\1747563238.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dm_last_30_days['DM Returned Item Cost'] = df_dm_last_30_days.apply(lambda x: 0.5 if x['Return/NOC Reason'] == 'REDEPOSITIED' else 1.2, axis=1)
C:\Users\pskotte\AppData\Local\Temp\ipykernel_15284\1747563238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ilms_last_30_days['ILMS Returned Item Cost'] = df_ilms_last_30_days.apply(lambda x: 0.5 if x['Return/NOC Reason'] == 'REDEPOSITIED

In [4]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Parameters for connection for DM
dm_server_name = 'rpt_ap_prd.internal.mcmcg.com'
dm_database_name = 'crs5_oltp'
dm_authentication = 'Trusted_Connection=yes' # Changed to use Windows Authentication

# Connection URI for DM
dm_conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={dm_server_name};'
    f'DATABASE={dm_database_name};'
    f'{dm_authentication};'
)

dm_quoted_conn_str = quote_plus(dm_conn_str)
dm_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={dm_quoted_conn_str}')

# SQL Query for Direct Mail Scheduled replaced with the new query
dm_query = """
SELECT DISTINCT 
    DATEADD(day, -1, CAST(schdld_pymnt_instnc_dttm AS DATE)) AS 'Payment Date', 
    COUNT(spi.schdld_pymnt_instnc_id) AS 'DM Transaction Count', 
    SUM(spi.schdld_pymnt_instnc_amnt) AS 'DM Scheduled Payment Amount'
FROM schdld_pymnt_smmry sps WITH (NOLOCK)
INNER JOIN schdld_pymnt_instnc spi WITH (NOLOCK) ON sps.schdld_pymnt_smmry_id = spi.schdld_pymnt_smmry_id 
INNER JOIN cnsmr_pymnt_instrmnt cpi WITH (NOLOCK) ON spi.schdld_pymnt_instnc_pymnt_instrmnt_id=cpi.cnsmr_pymnt_instrmnt_id 
INNER JOIN Ref_schdld_pymnt_stts_cd rsps WITH (NOLOCK) ON rsps.schdld_pymnt_stts_cd=spi.schdld_pymnt_instnc_stts_cd
WHERE schdld_pymnt_smmry_stts_cd = 9
AND schdld_pymnt_instnc_dttm BETWEEN CAST(DATEADD(DAY, -30, GETDATE()) AS DATE) AND CAST(DATEADD(DAY, 30, GETDATE()) AS DATE)
AND cpi.cnsmr_pymnt_instrmnt_typ_cd IN (16) -- Visa 14, MC 13, ACH 16
AND schdld_pymnt_stts_val_txt IN ('PENDING', 'MADE')
GROUP BY DATEADD(day, -1, CAST(schdld_pymnt_instnc_dttm AS DATE))
ORDER BY 'Payment Date' DESC
"""

# Execute the new query and store results in a pandas DataFrame for Direct Mail
DM_Scheduled = pd.read_sql_query(dm_query, dm_engine)

# Calculate 'Estimated Cost' by multiplying the 'Count' column by 0.7 for DM_Scheduled
DM_Scheduled['DM Estimated Approved Cost'] = DM_Scheduled['DM Transaction Count'] * 0.07

# Parameters for connection for ILMS remain the same
server_name = 'azwsynt00.sql.azuresynapse.net'
database_name = 'AZWSYNT00'
authentication = 'authentication=ActiveDirectoryIntegrated'

# Connection URI for ILMS
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server_name};'
    f'DATABASE={database_name};'
    f'{authentication};'
)

quoted_conn_str = quote_plus(conn_str)
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted_conn_str}')

# SQL Query for ILMS Scheduled remains the same
ilms_query = """
SELECT 
    CONVERT(DATE, dppd.duedate) AS 'Payment Date',
    SUM(dppd.paymentamount) AS 'ILMS Total Payment Amount',
    COUNT(*) AS 'ILMS Transaction Count'
FROM dw.ILMS_DIM_PAYMENT_PLAN_DETAIL dppd
WHERE dppd.paymentmethod = 'ACH'
AND dppd.duedate BETWEEN DATEADD(DAY, -30, GETDATE()) AND DATEADD(DAY, 30, GETDATE())
GROUP BY CONVERT(DATE, dppd.duedate)
ORDER BY 'Payment Date' ASC
"""

# Execute the ILMS query and store results in a pandas DataFrame
ILMS_Scheduled = pd.read_sql_query(ilms_query, engine)

# Calculate 'Estimated Cost' by multiplying the 'Count' column by 0.7 for ILMS_Scheduled
ILMS_Scheduled['ILMS Estimated Approved Cost'] = ILMS_Scheduled['ILMS Transaction Count'] * 0.07

print(DM_Scheduled)
print(ILMS_Scheduled)

   Payment Date  DM Transaction Count  DM Scheduled Payment Amount  \
0    2024-05-14                 38376                   2571049.93   
1    2024-05-13                  6225                    396021.70   
2    2024-05-12                  6106                    370342.20   
3    2024-05-11                  4397                    323915.21   
4    2024-05-10                  7102                    424300.67   
..          ...                   ...                          ...   
56   2024-03-19                 18241                   1195869.57   
57   2024-03-18                  5620                    412728.30   
58   2024-03-17                  7601                    519787.18   
59   2024-03-16                  5156                    339850.19   
60   2024-03-15                 10321                    660261.92   

    DM Estimated Approved Cost  
0                      2686.32  
1                       435.75  
2                       427.42  
3                       307

In [5]:
# Assuming DM_Scheduled and ILMS_Scheduled are the DataFrames you want to join on the 'Payment Date' column

joined_df = pd.merge(DM_Scheduled, ILMS_Scheduled, on='Payment Date', how='inner')

print(joined_df.to_string())

   Payment Date  DM Transaction Count  DM Scheduled Payment Amount  DM Estimated Approved Cost  ILMS Total Payment Amount  ILMS Transaction Count  ILMS Estimated Approved Cost
0    2024-05-14                 38376                   2571049.93                     2686.32                  106372.57                     991                         69.37
1    2024-05-13                  6225                    396021.70                      435.75                   81305.93                     862                         60.34
2    2024-05-12                  6106                    370342.20                      427.42                  110517.44                     919                         64.33
3    2024-05-11                  4397                    323915.21                      307.79                  112579.82                    1231                         86.17
4    2024-05-10                  7102                    424300.67                      497.14                  480348.5

In [6]:
# Assuming DM_Scheduled and ILMS_Scheduled are the DataFrames you want to join on the 'Payment Date' column
# and df_dm_daily_cost and df_ilms_daily_cost contain daily cost data to be joined

joined_df = pd.merge(DM_Scheduled, ILMS_Scheduled, on='Payment Date', how='inner')

# Joining daily cost DataFrames to the joined_df
joined_df = pd.merge(joined_df, df_dm_daily_cost, left_on='Payment Date', right_on='Report Date', how='left')
joined_df = pd.merge(joined_df, df_ilms_daily_cost, left_on='Payment Date', right_on='Report Date', how='left')

print(joined_df.to_string(index=False))

Payment Date  DM Transaction Count  DM Scheduled Payment Amount  DM Estimated Approved Cost  ILMS Total Payment Amount  ILMS Transaction Count  ILMS Estimated Approved Cost  DM Returned Item Cost  ILMS Returned Item Cost
  2024-05-14                 38376                   2571049.93                     2686.32                  106372.57                     991                         69.37                    NaN                      NaN
  2024-05-13                  6225                    396021.70                      435.75                   81305.93                     862                         60.34                    NaN                      NaN
  2024-05-12                  6106                    370342.20                      427.42                  110517.44                     919                         64.33                    NaN                      NaN
  2024-05-11                  4397                    323915.21                      307.79                  112579.

In [7]:
joined_df['DM Today Fees'] = joined_df['DM Estimated Approved Cost'] + (joined_df['DM Transaction Count'] * 0.07)
joined_df['ILMS Today Fees'] = joined_df['ILMS Estimated Approved Cost'] + (joined_df['ILMS Transaction Count'] * 0.07)

In [8]:
from datetime import datetime, timedelta

today_str = datetime.now().strftime('%Y-%m-%d')
today_date = datetime.now()
last_week_date = today_date - timedelta(days=6)  # Including today, hence 6 days back
last_month_date = today_date - timedelta(days=30)  # Last 30 days including today

# Ensure the 'Payment Date' is in the same datetime format before filtering
joined_df['Payment Date'] = pd.to_datetime(joined_df['Payment Date']).dt.strftime('%Y-%m-%d')

# Create a filtered DataFrame with today's date
TodaysACH = joined_df[joined_df['Payment Date'] == today_str].reset_index(drop=True)

# Filter for the last 7 days including today
LastWeekACH = joined_df[(joined_df['Payment Date'] >= last_week_date.strftime('%Y-%m-%d')) & 
                        (joined_df['Payment Date'] <= today_str)].reset_index(drop=True)

# Filter for the last 30 days including today
LastMonthACH = joined_df[(joined_df['Payment Date'] >= last_month_date.strftime('%Y-%m-%d')) & 
                         (joined_df['Payment Date'] <= today_str)].reset_index(drop=True)

print(TodaysACH)
print(LastWeekACH)
print(LastMonthACH)

  Payment Date  DM Transaction Count  DM Scheduled Payment Amount  \
0   2024-04-15                  9931                    668713.46   

   DM Estimated Approved Cost  ILMS Total Payment Amount  \
0                      695.17                  919001.81   

   ILMS Transaction Count  ILMS Estimated Approved Cost  \
0                    7760                         543.2   

   DM Returned Item Cost  ILMS Returned Item Cost  DM Today Fees  \
0                 5450.4                    151.2        1390.34   

   ILMS Today Fees  
0           1086.4  
  Payment Date  DM Transaction Count  DM Scheduled Payment Amount  \
0   2024-04-15                  9931                    668713.46   
1   2024-04-14                 32466                   2228074.10   
2   2024-04-13                  4341                    290738.32   
3   2024-04-12                  6700                    407881.64   
4   2024-04-11                 64020                   3618255.28   
5   2024-04-10              

In [9]:
joined_df.to_csv('C:\\Users\\pskotte\\Desktop\\Power Bi Data\\ACH Costs.csv', index=False)

In [10]:
TodaysACH.to_csv('C:\\Users\\pskotte\\Desktop\\Power Bi Data\\Todays ACH Costs.csv', index=False)

In [11]:
LastWeekACH.to_csv('C:\\Users\\pskotte\\Desktop\\Power Bi Data\\Last Week ACH Costs.csv', index=False)

In [12]:
LastMonthACH.to_csv('C:\\Users\\pskotte\\Desktop\\Power Bi Data\\Last Month ACH.csv', index=False)